# Resnet50

https://github.com/jonas-pettersson/fast-ai/blob/master/resnet50.ipynb

In [1]:
from theano.sandbox import cuda

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
import resnet50; reload(resnet50)
from resnet50 import Resnet50

In [4]:
#path = "data/dogscats/sample/"
path = "data/dogscats/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64
#batch_size=8

In [5]:
resnet_model = Resnet50(include_top=False).model

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [6]:
resnet_model.output_shape

(None, 2048, 7, 7)

In [7]:
resnet_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 3, 224, 224)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 230, 230)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 112, 112)  9472        zeropadding2d_1[0][0]            
___________________________________________________________________________________________

bn3a_branch2b (BatchNormalizatio (None, 128, 28, 28)   512         res3a_branch2b[0][0]             
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 128, 28, 28)   0           bn3a_branch2b[0][0]              
____________________________________________________________________________________________________
res3a_branch2c (Convolution2D)   (None, 512, 28, 28)   66048       activation_12[0][0]              
____________________________________________________________________________________________________
res3a_branch1 (Convolution2D)    (None, 512, 28, 28)   131584      activation_10[0][0]              
____________________________________________________________________________________________________
bn3a_branch2c (BatchNormalizatio (None, 512, 28, 28)   2048        res3a_branch2c[0][0]             
___________________________________________________________________________________________

res4a_branch2b (Convolution2D)   (None, 256, 14, 14)   590080      activation_23[0][0]              
____________________________________________________________________________________________________
bn4a_branch2b (BatchNormalizatio (None, 256, 14, 14)   1024        res4a_branch2b[0][0]             
____________________________________________________________________________________________________
activation_24 (Activation)       (None, 256, 14, 14)   0           bn4a_branch2b[0][0]              
____________________________________________________________________________________________________
res4a_branch2c (Convolution2D)   (None, 1024, 14, 14)  263168      activation_24[0][0]              
____________________________________________________________________________________________________
res4a_branch1 (Convolution2D)    (None, 1024, 14, 14)  525312      activation_22[0][0]              
___________________________________________________________________________________________

bn4e_branch2a (BatchNormalizatio (None, 256, 14, 14)   1024        res4e_branch2a[0][0]             
____________________________________________________________________________________________________
activation_35 (Activation)       (None, 256, 14, 14)   0           bn4e_branch2a[0][0]              
____________________________________________________________________________________________________
res4e_branch2b (Convolution2D)   (None, 256, 14, 14)   590080      activation_35[0][0]              
____________________________________________________________________________________________________
bn4e_branch2b (BatchNormalizatio (None, 256, 14, 14)   1024        res4e_branch2b[0][0]             
____________________________________________________________________________________________________
activation_36 (Activation)       (None, 256, 14, 14)   0           bn4e_branch2b[0][0]              
___________________________________________________________________________________________

bn5c_branch2a (BatchNormalizatio (None, 512, 7, 7)     2048        res5c_branch2a[0][0]             
____________________________________________________________________________________________________
activation_47 (Activation)       (None, 512, 7, 7)     0           bn5c_branch2a[0][0]              
____________________________________________________________________________________________________
res5c_branch2b (Convolution2D)   (None, 512, 7, 7)     2359808     activation_47[0][0]              
____________________________________________________________________________________________________
bn5c_branch2b (BatchNormalizatio (None, 512, 7, 7)     2048        res5c_branch2b[0][0]             
____________________________________________________________________________________________________
activation_48 (Activation)       (None, 512, 7, 7)     0           bn5c_branch2b[0][0]              
___________________________________________________________________________________________

In [8]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)　# shuffle must be False to compute convolution output
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size*2) # shuffle must be False to compute convolution output

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [10]:
trn_features = resnet_model.predict_generator(batches, batches.nb_sample) # batches.nb_sample == batches.n
save_array(model_path + 'train_resnet50_features.bc', trn_features)

In [13]:
val_features = resnet_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(model_path + 'valid_resnet50_features.bc', val_features)

In [14]:
trn_features = load_array(model_path+'train_resnet50_features.bc')
val_features = load_array(model_path+'valid_resnet50_features.bc')

In [14]:
trn_features.shape

(23000, 2048, 7, 7)

In [16]:
def get_fc_layers(p):
    return [
        BatchNormalization(axis=1, input_shape=resnet_model.output_shape[1:]), # input_shape=(2048, 7, 7)
        Flatten(),
        Dropout(p),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [17]:
model = Sequential(get_fc_layers(.5))

In [18]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 2048, 7, 7)    8192        batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 100352)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100352)        0           flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1024)          102761472   dropout_1[0][0]                  
___________________________________________________________________________________________

In [19]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(trn_features, trn_labels, nb_epoch=2, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/2
23000/23000 [==============================] - 39s - loss: 0.1111 - acc: 0.9720 - val_loss: 0.0588 - val_acc: 0.9820
Epoch 2/2
23000/23000 [==============================] - 38s - loss: 0.0277 - acc: 0.9912 - val_loss: 0.0605 - val_acc: 0.9870


## Global Average Pooling

In [22]:
def get_ap_layers(p):
    return [
        GlobalAveragePooling2D(input_shape=resnet_model.output_shape[1:]), # input_shape=(2048, 7, 7)
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [23]:
model = Sequential(get_ap_layers(.2))

In [24]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# very fast!
model.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 5s - loss: 0.0659 - acc: 0.9750 - val_loss: 0.0439 - val_acc: 0.9850
Epoch 2/3
23000/23000 [==============================] - 5s - loss: 0.0335 - acc: 0.9874 - val_loss: 0.0424 - val_acc: 0.9845
Epoch 3/3
23000/23000 [==============================] - 5s - loss: 0.0297 - acc: 0.9903 - val_loss: 0.0415 - val_acc: 0.9850


## Resnet Large

In [26]:
resnet_model = Resnet50(include_top=False, size=(400,400)).model

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 400, 400)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [27]:
resnet_model.output_shape

(None, 2048, 13, 13)

In [28]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size, target_size=(400,400)) # shuffle must be False
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size*2, target_size=(400,400)) # shuffle must be False
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [29]:
# maybe about 45 minutes to finish...
trn_features = resnet_model.predict_generator(batches, batches.nb_sample)
save_array(model_path + 'train_resnet50_lrg_features.bc', trn_features)

In [30]:
val_features = resnet_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(model_path + 'valid_resnet50_lrg_features.bc', val_features)

In [10]:
trn_features = load_array(model_path+'train_resnet50_lrg_features.bc')
val_features = load_array(model_path+'valid_resnet50_lrg_features.bc')

In [31]:
trn_features.shape

(23000, 2048, 13, 13)

In [32]:
# If you enconter with Memory Error, run this before loading trn_features, val_features. It may resolve...
model = Sequential(get_ap_layers(.01))

In [33]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 19s - loss: 0.0766 - acc: 0.9736 - val_loss: 0.0400 - val_acc: 0.9885
Epoch 2/3
23000/23000 [==============================] - 19s - loss: 0.0300 - acc: 0.9907 - val_loss: 0.0272 - val_acc: 0.9920
Epoch 3/3
23000/23000 [==============================] - 19s - loss: 0.0247 - acc: 0.9914 - val_loss: 0.0293 - val_acc: 0.9890
